# Factor Analysis using the CAPM and Fama-French Factor models

The main idea in Factor Analysis is to take a set of observed returns and decompose it into a set of explanatory returns.

We'll follow _Asset Management_ (Ang 2014, Oxford University Press) Chapter 10 and analyze the returns of Berkshire Hathaway.

First, we'll need the returns of Berkshire Hathaway which are contained in `data/brka_d_rets.csv`. Read it in as follows:

In [1]:
import pandas as pd

brka_d = pd.read_csv("data/brka_d_ret.csv", parse_dates=True, index_col=0)
brka_d.head()

,BRKA
DATE,
1990-01-02,-0.005764
1990-01-03,0.000000
1990-01-04,0.005797
1990-01-05,-0.005764
1990-01-08,0.000000


In [2]:
brka_d.tail()

,BRKA
DATE,
2018-12-24,-0.018611
2018-12-26,0.043200
2018-12-27,0.012379
2018-12-28,0.013735
2018-12-31,0.011236


Next, we need to convert these to monthly returns. The simplest way to do so is by using the `.resample` method, which allows you to run an aggregation function on each group of returns in a time series. We'll give it the grouping rule of 'M' which means _monthly_ (consult the `pandas`) documentation for other codes)

We want to compound the returns, and we already have the `compound` function in our toolkit, so let's load that up now, and then apply it to the daily returns.

In [3]:
import edhec_risk_kit_201 as erk

%load_ext autoreload
%autoreload 2

brka_m = brka_d.resample('M').apply(erk.compound).to_period('M')
brka_m.head()

,BRKA
DATE,
1990-01,-0.140634
1990-02,-0.030852
1990-03,-0.069204
1990-04,-0.003717
1990-05,0.067164


In [4]:
brka_m.to_csv("brka_m.csv") # for possible future use!

Next, we need to load the explanatory variables, which is the Fama-French monthly returns data set. Load that as follows:

In [5]:
fff = erk.get_fff_returns()
fff.head()

,Mkt-RF,SMB,HML,RF
1926-07,0.0296,-0.0230,-0.0287,0.0022
1926-08,0.0264,-0.0140,0.0419,0.0025
1926-09,0.0036,-0.0132,0.0001,0.0023
1926-10,-0.0324,0.0004,0.0051,0.0032
1926-11,0.0253,-0.0020,-0.0035,0.0031


Next, we need to decompose the observed BRKA 1990-May 2012 as in Ang(2014) into the portion that's due to the market and the rest that is not due to the market, using the CAPM as the explanatory model.

i.e.

$$ R_{brka,t} - R_{f,t} = \alpha + \beta(R_{mkt,t} - R_{f,t}) + \epsilon_t $$

We can use the `stats.api` for the linear regression as follows:

In [6]:
import statsmodels.api as sm
import numpy as np
brka_excess = brka_m["1990":"2012-05"] - fff.loc["1990":"2012-05", ['RF']].values
mkt_excess = fff.loc["1990":"2012-05",['Mkt-RF']]
exp_var = mkt_excess.copy()
exp_var["Constant"] = 1
lm = sm.OLS(brka_excess, exp_var).fit()

In [7]:
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   BRKA   R-squared:                       0.154
Model:                            OLS   Adj. R-squared:                  0.150
Method:                 Least Squares   F-statistic:                     48.45
Date:                Tue, 13 Aug 2019   Prob (F-statistic):           2.62e-11
Time:                        09:32:44   Log-Likelihood:                 388.47
No. Observations:                 269   AIC:                            -772.9
Df Residuals:                     267   BIC:                            -765.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Mkt-RF         0.5402      0.078      6.961      0.000       0.387       0.693
Constant       0.0061      0.004      1.744      0.082      -0.001       0.013
==============================================================================
Omnibus:                       45.698   Durbin-Watson:                   2.079
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              102.573
Skew:                           0.825   Prob(JB):                     5.33e-23
Kurtosis:                       5.535   Cond. No.                         22.2
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### The CAPM benchmark interpretation

This implies that the CAPM benchmark consists of 46 cents in T-Bills and 54 cents in the market. i.e. each dollar in the Berkshire Hathaway portfolio is equivalent to 46 cents in T-Bills and 54 cents in the market. Relative to this, the Berkshire Hathaway is adding (i.e. has $\alpha$ of) 0.61% _(per month!)_ although the degree of statistica significance is not very high.

Now, let's add in some additional explanatory variables, namely Value and Size.

In [8]:
exp_var["Value"] = fff.loc["1990":"2012-05",['HML']]
exp_var["Size"] = fff.loc["1990":"2012-05",['SMB']]
exp_var.head()

,Mkt-RF,Constant,Value,Size
1990-01,-0.0785,1,0.0087,-0.0129
1990-02,0.0111,1,0.0061,0.0103
1990-03,0.0183,1,-0.0290,0.0152
1990-04,-0.0336,1,-0.0255,-0.0050
1990-05,0.0842,1,-0.0374,-0.0257


In [9]:
lm = sm.OLS(brka_excess, exp_var).fit()
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   BRKA   R-squared:                       0.290
Model:                            OLS   Adj. R-squared:                  0.282
Method:                 Least Squares   F-statistic:                     36.06
Date:                Tue, 13 Aug 2019   Prob (F-statistic):           1.41e-19
Time:                        09:32:44   Log-Likelihood:                 412.09
No. Observations:                 269   AIC:                            -816.2
Df Residuals:                     265   BIC:                            -801.8
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Mkt-RF         0.6761      0.074      9.155      0.000       0.531       0.821
Constant       0.0055      0.003      1.679      0.094      -0.001       0.012
Value          0.3814      0.109      3.508      0.001       0.167       0.595
Size          -0.5023      0.101     -4.962      0.000      -0.702      -0.303
==============================================================================
Omnibus:                       42.261   Durbin-Watson:                   2.146
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               67.954
Skew:                           0.904   Prob(JB):                     1.75e-15
Kurtosis:                       4.671   Cond. No.                         37.2
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### The Fama-French Benchmark Interpretation

The alpha has fallen from .61% to about 0.55% per month. The loading on the market has moved up from 0.54 to 0.67, which means that adding these new explanatory factors did change things. If we had added irrelevant variables, the loading on the market would be unaffected.

We can interpret the loadings on Value being positive as saying that Hathaway has a significant Value tilt - which should not be a shock to anyone that follows Buffet. Additionally, the negative tilt on size suggests that Hathaway tends to invest in large companies, not small companies.

In other words, Hathaway appears to be a Large Value investor. Of course, you knew this if you followed the company, but the point here is that numbers reveal it!

The new way to interpret each dollar invested in Hathaway is: 67 cents in the market, 33 cents in Bills, 38 cents in Value stocks and short 38 cents in Growth stocks, short 50 cents in SmallCap stocks and long 50 cents in LargeCap stocks. If you did all this, you would still end up underperforming Hathaway by about 55 basis points per month.

We can now add the following code to the toolkit:

```python
import statsmodels.api as sm
def regress(dependent_variable, explanatory_variables, alpha=True):
    """
    Runs a linear regression to decompose the dependent variable into the explanatory variables
    returns an object of type statsmodel's RegressionResults on which you can call
       .summary() to print a full summary
       .params for the coefficients
       .tvalues and .pvalues for the significance levels
       .rsquared_adj and .rsquared for quality of fit
    """
    if alpha:
        explanatory_variables = explanatory_variables.copy()
        explanatory_variables["Alpha"] = 1
    
    lm = sm.OLS(dependent_variable, explanatory_variables).fit()
    return lm
```


## Exercise to the Student

I used this particular period because of the example in Ang (2014). However, I have provided data going up to 2018. Have the results held up? Are Buffet's tilts consistent over time?

In [10]:
result = erk.regress(brka_excess, mkt_excess)

In [11]:
result.params

Mkt-RF    0.540175
Alpha     0.006133
dtype: float64

In [12]:
result.tvalues

Mkt-RF    6.960550
Alpha     1.744449
dtype: float64

In [13]:
result.pvalues

Mkt-RF    2.622873e-11
Alpha     8.223148e-02
dtype: float64

In [14]:
result.rsquared_adj

0.15041804337083986

In [15]:
exp_var.head()

,Mkt-RF,Constant,Value,Size
1990-01,-0.0785,1,0.0087,-0.0129
1990-02,0.0111,1,0.0061,0.0103
1990-03,0.0183,1,-0.0290,0.0152
1990-04,-0.0336,1,-0.0255,-0.0050
1990-05,0.0842,1,-0.0374,-0.0257


In [16]:
erk.regress(brka_excess, exp_var, alpha=False).summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   BRKA   R-squared:                       0.290
Model:                            OLS   Adj. R-squared:                  0.282
Method:                 Least Squares   F-statistic:                     36.06
Date:                Tue, 13 Aug 2019   Prob (F-statistic):           1.41e-19
Time:                        09:32:44   Log-Likelihood:                 412.09
No. Observations:                 269   AIC:                            -816.2
Df Residuals:                     265   BIC:                            -801.8
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Mkt-RF         0.6761      0.074      9.155      0.000       0.531       0.821
Constant       0.0055      0.003      1.679      0.094      -0.001       0.012
Value          0.3814      0.109      3.508      0.001       0.167       0.595
Size          -0.5023      0.101     -4.962      0.000      -0.702      -0.303
==============================================================================
Omnibus:                       42.261   Durbin-Watson:                   2.146
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               67.954
Skew:                           0.904   Prob(JB):                     1.75e-15
Kurtosis:                       4.671   Cond. No.                         37.2
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [224]:
vw_49 = pd.read_csv("data/ind49_m_vw_rets.csv", header = 0, index_col=0, parse_dates=True, 
na_values=-99.99, sep=r'\s*,\s*')
vw_49.index = pd.to_datetime(vw_49.index, format="%Y%m")
vw_49.index = vw_49.index.to_period('M')
vw_49.head()

<ipython-input-224-d6b667d92f45>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  vw_49 = pd.read_csv("data/ind49_m_vw_rets.csv", header = 0, index_col=0, parse_dates=True,


,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,Clths,...,Boxes,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin,Other
1926-07,2.37,0.12,NaN,-5.19,1.29,8.65,2.50,50.21,-0.48,8.08,...,7.70,1.94,-23.79,0.07,1.87,4.61,-0.54,2.89,-4.85,5.20
1926-08,2.23,2.68,NaN,27.03,6.50,16.81,-0.76,42.98,-3.58,-2.51,...,-2.38,4.88,5.39,-0.75,-0.13,11.83,2.57,5.30,-0.57,6.76
1926-09,-0.57,1.58,NaN,4.02,1.26,8.33,6.42,-4.91,0.73,-0.51,...,-5.54,0.06,-7.87,0.25,-0.56,-1.75,0.72,-3.06,-3.14,-3.86
1926-10,-0.46,-3.68,NaN,-3.31,1.06,-1.40,-5.09,5.37,-4.68,0.12,...,-5.08,-2.64,-15.38,-2.20,-4.11,-11.82,-4.28,-5.74,2.07,-8.49
1926-11,6.75,6.26,NaN,7.29,4.55,0.00,1.82,-6.40,-0.54,1.87,...,3.84,1.60,4.67,6.52,4.33,-2.97,3.58,2.21,4.92,4.00


In [253]:
import statsmodels.api as sm
import numpy as np
import edhec_risk_kit_206 as erk

fff = erk.get_fff_returns()
def calcCAMP(startYear, endYear, industry, threeFactor=False):
    
    returnData = vw_49[startYear:endYear]
    r = returnData[[industry]]
    r = r/100

    data_excess = r - fff.loc[startYear:endYear, ['RF']].values
    mkt_excess = fff.loc[startYear:endYear,['Mkt-RF']]
    exp_var = mkt_excess.copy()
    exp_var["Constant"] = 1
    if threeFactor:
        exp_var["Size"] = fff.loc[startYear:endYear,['SMB']]
        exp_var["Value"] = fff.loc[startYear:endYear,['HML']]
    lm = sm.OLS(data_excess, exp_var).fit()
    return lm.params;

**CAPM (Single Factor) Beta when evaluated over the entire period (1991-2018) of Beer**

_Load the 49 Value Weighted industry portfolio returns_

In [254]:
calcCAMP("1991", "2018", "Beer")["Mkt-RF"]

0.5295417263363026

**CAPM Beta when evaluated over the entire period (1991-2018) of Steel**

In [241]:
calcCAMP("1991", "2018", "Steel")["Mkt-RF"]

1.5546095998085294

**CAPM Beta when evaluated over the 2013-2018 (both included) period of Beer**



In [242]:
calcCAMP("2013", "2018", "Beer")["Mkt-RF"]

0.585960389056597

**CAPM Beta when evaluated over the 2013-2018 (both included) period of Steel**

In [243]:
calcCAMP("2013", "2018", "Steel")["Mkt-RF"]

1.4169446523353808

**49 industries had the highest CAPM Beta when evaluated over the 1991-1993 (both included) period**

In [255]:
industryVals = {}

for col in vw_49.columns:
    industryVals[col] = calcCAMP("1991", "1993", col)["Mkt-RF"]
    
pd.DataFrame(industryVals.items(), columns=['Industry', 'CAPM']).sort_values("CAPM", ascending=False)

,Industry,CAPM
10,Hlth,1.615195
35,Softw,1.608340
17,Cnstr,1.520882
9,Clths,1.516927
11,MedEq,1.473925
48,Other,1.454826
44,Banks,1.359233
4,Smoke,1.323776
2,Soda,1.319035
36,Chips,1.272863


In [256]:
industryVals = {}

for col in vw_49.columns:
    industryVals[col] = calcCAMP("1991", "2018", col, threeFactor=True)['Size']
pd.DataFrame(industryVals.items(), columns=['Industry', 'Size']).sort_values("Size", ascending=False)

,Industry,Size
19,FabPr,0.688508
46,RlEst,0.687456
15,Txtls,0.535961
28,Coal,0.534373
18,Steel,0.493311
37,LabEq,0.465178
14,Rubbr,0.448314
17,Cnstr,0.408917
20,Mach,0.353644
5,Toys,0.329862


In [247]:
industryVals = {}

for col in vw_49.columns:
    industryVals[col] = calcCAMP("1991", "2018", col, threeFactor=True)['Value']
pd.DataFrame(industryVals.items(), columns=['Industry', 'Value']).sort_values("Value", ascending=False)

,Industry,Size
15,Txtls,0.983560
46,RlEst,0.896931
44,Banks,0.839294
22,Autos,0.797194
24,Ships,0.773196
16,BldMt,0.608665
17,Cnstr,0.607808
25,Guns,0.591323
45,Insur,0.570370
27,Mines,0.565671


In [257]:
industryVals = {}

for col in vw_49.columns:
    industryVals[col] = calcCAMP("1991", "2018", col, threeFactor=True)['Mkt-RF']
pd.DataFrame(industryVals.items(), columns=['Industry', 'Mkt-RF']).sort_values("Mkt-RF", ascending=False)

,Industry,Mkt-RF
18,Steel,1.509446
47,Fin,1.413037
36,Chips,1.394294
34,Hardw,1.334896
22,Autos,1.329642
6,Fun,1.326548
20,Mach,1.280466
21,ElcEq,1.263787
35,Softw,1.255132
15,Txtls,1.236172


In [258]:
industryVals = {}

for col in vw_49.columns:
    industryVals[col] = calcCAMP("1991", "2018", col)["Constant"]
pd.DataFrame(industryVals.items(), columns=['Industry', 'Constant']).sort_values("Constant", ascending=False)

,Industry,Constant
25,Guns,0.006857
4,Smoke,0.005304
2,Soda,0.004421
3,Beer,0.004294
43,Meals,0.003963
24,Ships,0.003796
23,Aero,0.003755
30,Util,0.003568
0,Agric,0.003332
11,MedEq,0.003259


In [198]:
industryVals = {}

for col in vw_49.columns:
    industryVals[col] = calcCAMP("1991", "2018", col)["Constant"]
pd.DataFrame(industryVals.items(), columns=['Industry', 'Size']).sort_values("Size", ascending=False)

,Industry,Size
25,Guns,0.005143
35,Softw,0.004830
4,Smoke,0.004717
3,Beer,0.004493
12,Drugs,0.004103
2,Soda,0.003340
11,MedEq,0.003316
43,Meals,0.003265
0,Agric,0.002848
30,Util,0.002820
